### 接下來定義一個爬蟲函數，這個函數的主要工作為：
* (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
* (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
* (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [38]:
import requests , re
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
import os
input_kw = '初音未來'
utf8_url = repr(input_kw.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_kw, utf8_url[2:-1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    



初音未來: %E5%88%9D%E9%9F%B3%E6%9C%AA%E4%BE%86
/wiki/%E5%88%9D%E9%9F%B3%E6%9C%AA%E4%BE%86


In [11]:
url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())

初音未來[註 1][註 2]（日語：初音 ミク／はつね ミク Hatsune Miku，中文圈通常簡稱為「初音」）是Crypton Future Media（下略作「Crypton」）以Yamaha的VOCALOID 2語音合成引擎為基礎，開發販售的虛擬女性歌手，角色主唱系列的第一作、VSTi（日語：Virtual Studio Technology）規格的電子樂器；或此軟體的印象角色（這只是軟體的象徵，不會在實際使用時出現）。2007年8月31日發售，開放價格，官方估計系列軟體實際價格約15,750日圓。原只可用於Microsoft Windows，2008年3月19日隨CrossOver Mac 6.1發表而可用於macOS[4]，2014年發布的VOCALOID 3版本的軟體和2016年發布的VOCALOID 4版本的軟體已經原生支援macOS。在GNU/Linux系統上透過修改過的Wine也能夠正確執行初音未來[5]。發售後大受歡迎，並出現大量用戶製作的翻唱歌曲、原創曲等。官方表示初音未來擅長由1980年代至最新的流行歌曲。

初音未來除了吸引新一代和從不認識電子音樂的人接觸電子音樂創作，令更多人認識電子音樂製作，亦令約20年前創作熱潮時的老手「回歸」。電子音樂創作再度捲起熱潮，也刺激到音樂創作以外的繪畫、動畫創作。雖然初音未來並非第一個可以模仿人類歌唱的軟體，但像真度比以往同類軟體高，因而引起的熱潮帶來業餘音樂製作的革命，促造日本的消費者自組媒體發展。

軟體使用了Yamaha的VOCALOID 2、VOCALOID 3及VOCALOID 4的語音合成引擎，把人類的聲音錄音並合成為酷似真正的歌聲。只需輸入音調、歌詞則可發出聲音，亦可以調整震音、音速等的「感情參數」，最多能夠16人合唱，亦支援即時演奏、對應ReWire。製作完成後會以WAV格式輸出，但軟體本身只可做出歌唱部分，伴奏音聲需要使用其他音樂軟體合成。歌詞輸入能辨認平假名、片假名和羅馬字，但は、へ作為助詞時無法轉換為わ、え的發音，う、い作為長音時也無法變成お、え的發音，亦不能對應促音、漢字，需要自行修改。

軟體並非設計給初學者，初接觸電子音樂製作的用戶需要一段時間適應[7]，因此在2ch等地方也出現不少以此類用戶為物件的教學討論串。大阪電氣通訊大學（日語：大阪電気通信大学）甚至於2007年10月1

In [19]:
for ext_link in content:
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

外部連結: [Crypton Future Media] /wiki/Crypton_Future_Media
外部連結: [Yamaha] /wiki/%E5%B1%B1%E8%91%89%E5%85%AC%E5%8F%B8
外部連結: [VOCALOID 2] /wiki/VOCALOID
外部連結: [語音合成] /wiki/%E8%AA%9E%E9%9F%B3%E5%90%88%E6%88%90
外部連結: [歌手] /wiki/%E6%AD%8C%E6%89%8B
外部連結: [角色主唱系列] /wiki/%E8%A7%92%E8%89%B2%E4%B8%BB%E5%94%B1%E7%B3%BB%E5%88%97
外部連結: [日圓] /wiki/%E6%97%A5%E5%9C%93
外部連結: [Microsoft Windows] /wiki/Microsoft_Windows
外部連結: [CrossOver Mac] /wiki/CrossOver_Mac
外部連結: [macOS] /wiki/MacOS
外部連結: [VOCALOID 3] /wiki/VOCALOID
外部連結: [VOCALOID 4] /wiki/VOCALOID
外部連結: [GNU/Linux] /wiki/GNU/Linux
外部連結: [Wine] /wiki/Wine
外部連結: [Yamaha] /wiki/%E5%B1%B1%E8%91%89%E6%A0%AA%E5%BC%8F%E6%9C%83%E7%A4%BE
外部連結: [VOCALOID 2] /wiki/VOCALOID
外部連結: [語音合成] /wiki/%E8%AA%9E%E9%9F%B3%E5%90%88%E6%88%90
外部連結: [ReWire] /wiki/ReWire
外部連結: [WAV] /wiki/WAV
外部連結: [平假名] /wiki/%E5%B9%B3%E5%81%87%E5%90%8D
外部連結: [片假名] /wiki/%E7%89%87%E5%81%87%E5%90%8D
外部連結: [羅馬字] /wiki/%E7%BE%85%E9%A6%AC%E5%AD%97
外部連結: [は] /wiki/%E3%81%AF
外部連結: [へ] /wiki/%E3%81%B

In [52]:
def wiki_clawer(input_kw):
    utf8_url = repr(input_kw.encode('UTF-8')).upper()
    utf8_url = utf8_url.replace('\\X','%')
    url = 'https://zh.wikipedia.org'+'/wiki/' + utf8_url[2:-1]
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#         print(paragraph.get_text())
    with open('{}.txt'.format(input_kw),'w',encoding="utf-8") as fh:
        for paragraph in content:
            fh.write(paragraph.get_text())
    kw_list = list()
    for ext_link in content:
        a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
        if len(a_tag) > 0:
            for link_string in a_tag:
                a_link = link_string["href"]       # 外部連結的網址
                a_keyword = link_string.get_text()  # 外部連結的中文名稱
#                 print("外部連結: [%s] %s" % (a_keyword, a_link))
                kw_list.append(a_keyword)
    return kw_list
kw_input = '初音未來'
kw_list = wiki_clawer(kw_input)[]
for kw in kw_list:
    wiki_clawer(kw)

FileNotFoundError: [Errno 2] No such file or directory: 'GNU/Linux.txt'